In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Задание 1. Подбор оптимальных гиперпараметров для ALS

- Попробуйте улучшить базовый вариант ALS, изменяя следующие параметры
  - regularization, iterations
  - factors
  - Вес (TF_IDF, BM25  взвешивание)
  
- Посчитайте метрики (Precision@5, MAP@5) для разных наборов гиперпараметров и выберете лучший набор

**Функции для метрик Precision@5, MAP@5**

In [5]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [6]:
def precision_at_k_(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision



In [7]:
def map_k(recommended_list, bought_list, k=5):
    
    # сделать дома
    s = len(bought_list)
    list_ap_k = [ap_k(recommended_list[i], bought_list[i], k) for i in np.arange(s)]
    print(list_ap_k)
    result = sum(list_ap_k)/s
    
    return result

**Функция для получения рекомендательной модели**

In [8]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

**Отбор 5000 самых популярных товаров**

In [9]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [10]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\tatya\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

**ALS**

In [12]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 4.52 s


In [13]:
[id_to_itemid[rec[0]] for rec in recs]

[1133018, 999999, 1106523, 1082185, 5569230]

In [14]:
%%time
    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

Wall time: 24.4 s


0.16160626836434638

In [15]:
result.head(2)

,user_id,actual,als
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1033142, 5569374, 979707, 995242, 1005186]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5568378, 5569327, 938700, 1106523, 965766]"


In [16]:
#parameters = {'factors':[10, 100, 1000], 'regularization': [0, 0.01, 0.001, 0.0001], 'iterations': [10, 50, 100]}
#GridSearchCV(AlternatingLeastSquares(), parameters)

In [17]:
model.item_factors.shape

(5001, 100)

In [18]:
fast_recs = model.user_factors @ model.item_factors.T 
fast_recs.shape


(2499, 5001)

In [19]:
fast_recs[0,:]

array([ 0.0029486 ,  0.01605161, -0.0632089 , ...,  0.03615618,
        0.00988807, -0.15967126], dtype=float32)

In [20]:
%%time
recommendations = model.recommend_all(N=5, 
                                      user_items=csr_matrix(user_item_matrix).tocsr(),
                                      filter_already_liked_items=True, 
                                      filter_items=None, 
                                      recalculate_user=True,
                                      show_progress=True,
                                      batch_size=500)
recommendations

  0%|          | 0/2499 [00:00<?, ?it/s]

Wall time: 20.3 s


array([[3616, 2696,  832, 3947,  661],
       [2148, 2307,  391,  215, 4346],
       [1516, 3388, 1242, 3029, 2434],
       ...,
       [4346, 2148, 3587, 3133, 2307],
       [2847,  300, 1312, 3704, 1927],
       [ 558, 3688, 1328, 2454,  575]], dtype=int32)

In [21]:
recommendations.shape

(2499, 5)

In [22]:
model.item_factors.shape


(5001, 100)

In [23]:
%%time

summary_data = pd.DataFrame(data=[[0, 0, 0]], 
                            columns=['factors', 'regularization', 'precision'])

count = 0
factors = [25, 50, 75, 100]
regularization = [0.01, 0.001, 0.0001]

for fact in factors:
    for reg in regularization:
        name = f"ALS_{fact}_{reg}"         
        model = AlternatingLeastSquares(factors=fact, 
                                            regularization=reg,
                                            iterations=15, 
                                            calculate_training_loss=True, 
                                            num_threads=10,
                                            use_gpu=False)

        model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                      show_progress=False)

        recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                                   user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                                   N=5, # кол-во рекомендаций 
                                   filter_already_liked_items=False, 
                                   filter_items=None, 
                                   recalculate_user=True)
    
        result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        score = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
            
        summary_data.loc[count, ['factors', 'regularization', 'precision']] = [fact, reg, score]

        count += 1

Wall time: 3min 36s


In [24]:
summary_data

,factors,regularization,precision
0,25.0,0.0100,0.164251
1,25.0,0.0010,0.164251
2,25.0,0.0001,0.166699
3,50.0,0.0100,0.166503
4,50.0,0.0010,0.165230
5,50.0,0.0001,0.163369
6,75.0,0.0100,0.166405
7,75.0,0.0010,0.162390
8,75.0,0.0001,0.164349
9,100.0,0.0100,0.162586


In [25]:
summary_data.sort_values('precision', ascending=False)

,factors,regularization,precision
2,25.0,0.0001,0.166699
3,50.0,0.0100,0.166503
6,75.0,0.0100,0.166405
4,50.0,0.0010,0.165230
8,75.0,0.0001,0.164349
0,25.0,0.0100,0.164251
1,25.0,0.0010,0.164251
5,50.0,0.0001,0.163369
9,100.0,0.0100,0.162586
7,75.0,0.0010,0.162390


Наилучший результат получился при следующем наборе параметров: factors = 25, regularization = 0.0001, и составил - 0.166699